In [ ]:
!pip install certifi

In [ ]:
!pip install urllib3[secure]

In [11]:
# Worked
#  Cluster number, e.g. 100000
cluster  = '10000'

# Cluster username
username = 'yerrags'

# Cluster password
password = 'venkateshudu123'

# file path in HDFS
filepath = 'user/yerrags/test_space/cust.csv'

In [12]:
import pandas as pd
        
def read_csv_lines(lines, is_first_chunk = False):
    ''' returns a pandas dataframe '''
    
    if is_first_chunk:
        # you will want to set the header here if your datafile has a header record
        return pd.read_csv(lines, sep='|', header=None)
    else:
        return pd.read_csv(lines, sep='|', header=None)


In [16]:
# host = 'ehaasp-{0}-mastermanager.bi.services.bluemix.net'.format(cluster)

host = 'bi-hadoop-prod-4145.bi.services.us-south.bluemix.net'


In [14]:
import requests
import numpy as np
import sys
import datetime

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

print('SCRIPT START: {0}'.format(datetime.datetime.now()))

chunk_size = 10000000 # Read in 100 Mb chunks

url = "https://{0}:8443/gateway/default/webhdfs/v1/{1}?op=OPEN".format(host, filepath)

# note SSL verification is been disabled
r = requests.get(url, 
                 auth=(username, password), 
                 verify=False, 
                 allow_redirects=True, 
                 stream=True)

r.raise_for_status()

df = None
chunk_num = 1
remainder = ''
for chunk in r.iter_content(chunk_size):
    
    if chunk: # filter out keep-alive new chunks
        
        # Show progress by printing a dot - useful when chunk size is quite small
        # sys.stdout.write('.')
        # sys.stdout.flush()

        txt = remainder + chunk
        if '\n' in txt:
            [lines, remainder] = txt.rsplit('\n', 1)
        else:
            lines = txt

        if chunk_num == 1:
            pdf = read_csv_lines(StringIO(lines), True)
            df = sqlContext.createDataFrame(pdf)
        else:
            pdf = read_csv_lines(StringIO(lines), False)
            df2 = sqlContext.createDataFrame(pdf)
            
            df = df.sql_ctx.createDataFrame(
                    df._sc.union([df.rdd, df2.rdd]), df.schema
                    )
            
        print('Imported chunk: {0} record count: {1} df count: {2}'.format(chunk_num, len(pdf), df.count()))
            
        chunk_num = chunk_num + 1
        
print '\nTotal record import count: {0}'.format(df.count())

print('SCRIPT END: {0}'.format(datetime.datetime.now()))

df.cache()

SCRIPT START: 2016-09-15 11:38:58.709639


/usr/local/src/bluemix_jupyter_bundle.v14/notebook/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/usr/local/src/bluemix_jupyter_bundle.v14/notebook/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


Imported chunk: 1 record count: 4 df count: 4

Total record import count: 4
SCRIPT END: 2016-09-15 11:39:08.525407


DataFrame[0: string, 1: string, 2: string]

In [15]:
df.show()


+-----+--------+---+
|    0|       1|  2|
+-----+--------+---+
|Empid|    Name| Id|
|  001|Sreedhar| 98|
|  002|   Latha| 99|
|  003|Architha| 90|
+-----+--------+---+

